In [1]:
import DES
import numpy as np
import simpy as sip
import pandas as pd

In [2]:
max_iter = 10000
n = [1, 2, 4]
mu = 9/10
lamd = 1
# print(f"rho = {lamd/(1/mu*n)}")
a = np.random.exponential
b = np.random.exponential
queue_type = sip.Resource

for n_servers in n:
    t = f"{n_servers}_servers"
    print(t)

    for i in range(100):
        DES.loading_bar(i)
        DES.main_des(max_iter, n_servers, mu, lamd, a, b, queue_type, t)
    print("")

1_servers
Running simulations: ██████████

2_servers
Running simulations: ██████████

4_servers
Running simulations: ██████████



In [7]:
n_servers = 1
t = f"{n_servers}_servers"
res = pd.read_csv(f"logged_data/{t}.csv")
# sum(res["Queue_length"])
np.mean(res["wait_time"]), np.std(res["wait_time"])

(8.080837654720314, 8.857296003210452)

In [8]:
n_servers = 2
t = f"{n_servers}_servers"
res = pd.read_csv(f"logged_data/{t}.csv")
# sum(res["Queue_length"])
np.mean(res["wait_time"]), np.std(res["wait_time"])

(0.22888034672233146, 0.5675294220898974)

In [9]:
n_servers = 4
t = f"{n_servers}_servers"
res = pd.read_csv(f"logged_data/{t}.csv")
# sum(res["Queue_length"])
np.mean(res["wait_time"]), np.std(res["wait_time"])

(0.004046473711472952, 0.04661692371860618)